<a href="https://colab.research.google.com/github/anandhc6/Assignment-3/blob/main/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import math
import matplotlib.ticker as ticker
import numpy as np
from random import randrange 
from google.colab import files
import pandas as pd
import random
import tensorflow as tf
from tensorflow import keras
from keras import backend
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf 'dakshina_dataset_v1.0.tar'

--2022-05-06 19:25:48--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.97.128, 142.251.107.128, 173.194.210.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   225MB/s    in 9.1s    

2022-05-06 19:25:57 (212 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [ ]:
batch_size = 64  # Batch size for training.
# epochs = 10  # Number of epochs to train for.
# latent_dim = 256  # Latent dimensionality of the encoding space. #hidden states hyperparameter
# Path to the data txt file on disk.
train_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv"
val_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv"
# open and save the files to lists
with open(train_data, "r", encoding="utf-8") as f:
    train_lines = f.read().split("\n")
with open(val_data, "r", encoding="utf-8") as f:
    val_lines = f.read().split("\n")
# popping the last element of all the lists since it is empty character
train_lines.pop()
val_lines.pop()
random.shuffle(train_lines)
print(train_lines[0:2])


['பரிந்துரை\tparinthurai\t2', 'எடுப்பதில்லை\tyeduppathillai\t1']


In [ ]:
# embedding train model
def embedding_train(train_lines):

    input_texts = []
    target_texts = []
    input_characters = set()
    target_characters = set()
    # go through the train lines and split them into 3 and save input and target
    for line in train_lines[: (len(train_lines) - 1)]:
        # because we want english to devanagiri conversion
        target_text, input_text, _ = line.split("\t")
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = "\t" + target_text + "\n"
        # append it to the main input texts list
        input_texts.append(input_text)
        # append it to the main target texts list
        target_texts.append(target_text)
        # to find the number of unique characters in both
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
    # add the space character to both
    input_characters.add(" ")
    target_characters.add(" ")
    # sort it
    input_characters = sorted(list(input_characters))
    target_characters = sorted(list(target_characters))
    # find the number
    num_encoder_tokens = len(input_characters)
    num_decoder_tokens = len(target_characters)
    # find the maximum length of input word and target word
    max_encoder_seq_length = max([len(txt) for txt in input_texts])
    max_decoder_seq_length = max([len(txt) for txt in target_texts])
    
    print("Number of samples:", len(input_texts))
    print("Number of unique input tokens:", num_encoder_tokens)
    print("Number of unique output tokens:", num_decoder_tokens)
    print("Max sequence length for inputs:", max_encoder_seq_length)
    print("Max sequence length for outputs:", max_decoder_seq_length)
    # create an index
    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
   
    # create an 0 array for encoder input size of (input_texts,max_seqlen,tokens)
    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length), dtype="float32")
    # create decoder input
    decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length), dtype="float32")
    # create decoder target
    decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
    # for each sample convert it into character encoding i.e. if
    # at that position a character is present then encode the index of that character there
    # this is done for both encoder and decoder input data for further word embedding
    # but target data is one hot encoded.
    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t] = input_token_index[char]
        # remaining positions set as empty space
        encoder_input_data[i, t + 1:] = input_token_index[" "]
        # similarly do for decoder data
        for t, char in enumerate(target_text):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t] = target_token_index[char]
            # check if t >0 since decoder targer data is ahead
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
        # append both the remaining positions of both the datas with empty space
        decoder_input_data[i, t + 1:] = target_token_index[" "]
        decoder_target_data[i, t:, target_token_index[" "]] = 1.0

    return encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_tokens,num_decoder_tokens,input_token_index,target_token_index,max_encoder_seq_length,max_decoder_seq_length


In [ ]:
# embedding validation data
# for validation data, almost same
def embedding_val(val_lines,num_decoder_tokens,input_token_index,target_token_index):
    val_input_texts = []
    val_target_texts = []
    
    for line in val_lines[: (len(val_lines) - 1)]:
        target_text, input_text, _ = line.split("\t")
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = "\t" + target_text + "\n"
        val_input_texts.append(input_text)
        val_target_texts.append(target_text)
    val_max_encoder_seq_length = max([len(txt) for txt in val_input_texts])
    val_max_decoder_seq_length = max([len(txt) for txt in val_target_texts])
    val_encoder_input_data = np.zeros(
        (len(val_input_texts), val_max_encoder_seq_length), dtype="float32")
    val_decoder_input_data = np.zeros(
        (len(val_input_texts), val_max_decoder_seq_length), dtype="float32")
    val_decoder_target_data = np.zeros(
        (len(val_input_texts), val_max_decoder_seq_length, num_decoder_tokens), dtype="float32")
    for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
        for t, char in enumerate(input_text):
            val_encoder_input_data[i, t] = input_token_index[char]
        val_encoder_input_data[i, t + 1:] = input_token_index[" "]
        for t, char in enumerate(target_text):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            val_decoder_input_data[i, t] = target_token_index[char]
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
        val_decoder_input_data[i, t + 1:] = target_token_index[" "]
        val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

    return val_encoder_input_data,val_decoder_input_data,val_decoder_target_data,target_token_index,val_target_texts


In [ ]:
#Embedding data
encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_tokens,num_decoder_tokens,input_token_index,target_token_index,max_encoder_seq_length,max_decoder_seq_length = embedding_train(train_lines)
val_encoder_input_data,val_decoder_input_data,val_decoder_target_data,target_token_index,val_target_texts = embedding_val(val_lines,num_decoder_tokens,input_token_index,target_token_index)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


Number of samples: 68217
Number of unique input tokens: 27
Number of unique output tokens: 49
Max sequence length for inputs: 30
Max sequence length for outputs: 28


In [ ]:
#yoursssss

batch_size = 64  # Batch size for training.
epochs = 2  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space. #hidden states hyperparameter
# Path to the data txt file on disk.
train_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv"
val_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv"
# open and save the files to lists
with open(train_data, "r", encoding="utf-8") as f:
    train_lines = f.read().split("\n")
with open(val_data, "r", encoding="utf-8") as f:
    val_lines = f.read().split("\n")
# popping the last element of all the lists since it is empty character
train_lines.pop()
val_lines.pop()
random.shuffle(train_lines)
print(train_lines[0:2])

# embedding pre processing
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
# go through the train lines and split them into 3 and save input and target
for line in train_lines[: (len(train_lines) - 1)]:
    # because we want english to devanagiri conversion
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    # append it to the main input texts list
    input_texts.append(input_text)
    # append it to the main target texts list
    target_texts.append(target_text)
    # to find the number of unique characters in both
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
# add the space character to both
input_characters.add(" ")
target_characters.add(" ")
# sort it
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
# find the number
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# find the maximum length of input word and target word
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
# create an index
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
print((input_token_index))
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print((target_token_index))
# create an 0 array for encoder input size of (input_texts,max_seqlen,tokens)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length), dtype="float32"
)
# create decoder input
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length), dtype="float32"
)
# create decoder target
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
# for each sample convert it into character encoding i.e. if
# at that position a character is present then encode the index of that character there
# this is done for both encoder and decoder input data for further word embedding
# but target data is one hot encoded.
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[char]
    # remaining positions set as empty space
    encoder_input_data[i, t + 1:] = input_token_index[" "]
    # similarly do for decoder data
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[char]
        # check if t >0 since decoder targer data is ahead
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    # append both the remaining positions of both the datas with empty space
    decoder_input_data[i, t + 1:] = target_token_index[" "]
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


# for validation data,
val_input_texts = []
val_target_texts = []
for line in val_lines[: (len(val_lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    val_input_texts.append(input_text)
    val_target_texts.append(target_text)
val_max_encoder_seq_length = max([len(txt) for txt in val_input_texts])
val_max_decoder_seq_length = max([len(txt) for txt in val_target_texts])
val_encoder_input_data = np.zeros(
    (len(val_input_texts), val_max_encoder_seq_length), dtype="float32"
)
val_decoder_input_data = np.zeros(
    (len(val_input_texts), val_max_decoder_seq_length), dtype="float32"
)
val_decoder_target_data = np.zeros(
    (len(val_input_texts), val_max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t] = input_token_index[char]
    val_encoder_input_data[i, t + 1:] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    val_decoder_input_data[i, t + 1:] = target_token_index[" "]
    val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


In [ ]:
def seq2seq(embedding_size, n_encoder_tokens, n_decoder_tokens, n_encoder_layers,
                 n_decoder_layers, latent_dimension, cell_type,
                 target_token_index, max_decoder_seq_length, reverse_target_char_index,
                 dropout,encoder_input_data, decoder_input_data,
                decoder_target_data,
                batch_size,epochs):
  encoder_inputs = keras.Input(shape=(None,), name='encoder_input')
  # word embedding layer
  encoder = None
  encoder_outputs = None
  state_h = None
  state_c = None
  e_layer=n_encoder_layers

  if cell_type=="RNN":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,
                                             name='encoder_embedding')(encoder_inputs)
    encoder = keras.layers.SimpleRNN(latent_dimension, return_state=True, return_sequences=True,
                                             name='encoder_hidden_1', dropout=dropout)
    print("Embed done")
    encoder_outputs, state_h = encoder(embed)
    for i in range(2,e_layer+1):
      layer_name = ('encoder_hidden_%d') % i
      print("Starting 2nd")
      encoder = keras.layers.SimpleRNN(latent_dimension, return_state=True, return_sequences=True,
                                                 name=layer_name, dropout=dropout)
      print("Ending 2nd")

      encoder_outputs, state_h = encoder(encoder_outputs, initial_state=[state_h])
    encoder_states = None
    encoder_states = [state_h]

    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(
        decoder_inputs)
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.SimpleRNN(latent_dimension, return_sequences=True, return_state=True,
                                             name='decoder_hidden_1', dropout=dropout)
    # all decoders the initial state is encoder last state of last layer
    decoder_outputs, _ = decoder(embed_dec, initial_state=encoder_states)
    for i in range(2,d_layer+1):
      layer_name = 'decoder_hidden_%d' % i
      decoder = keras.layers.SimpleRNN(latent_dimension, return_sequences=True, return_state=True,
                                                 name=layer_name, dropout=dropout)
      decoder_outputs, _ = decoder(decoder_outputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax", name='decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    model.compile(
          optimizer="rmsprop", loss="categorical_crossentropy",
          metrics=['accuracy'])#, metrics=[my_metric]                 

    # earlystopping = EarlyStopping(
    #     monitor="val_loss", min_delta=0.01, patience=5, verbose=2, mode="min")

    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=WandbCallback()
      )
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc = model.get_layer(
              'encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [state_h_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  # input_2
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []

    for j in range(1, n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs, state_h_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec]
        decoder_states_inputs += current_states_inputs
    decoder_dense = model.get_layer('decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model(
        [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
    )
    return encoder_model, decoder_model

  elif cell_type=="GRU":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,
                                             name='encoder_embedding')(encoder_inputs)
    encoder = keras.layers.GRU(latent_dimension, return_state=True, return_sequences=True,
                                             name='encoder_hidden_1', dropout=dropout)
    encoder_outputs, state_h = encoder(embed)
    for i in range(2,e_layer+1):
      layer_name = ('encoder_hidden_%d') % i
      encoder = keras.layers.GRU(latent_dimension, return_state=True, return_sequences=True,
                                                 name=layer_name, dropout=dropout)
      encoder_outputs, state_h = encoder(encoder_outputs, initial_state=[state_h])
    encoder_states = None
    encoder_states = [state_h]

    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(
        decoder_inputs)
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.GRU(latent_dimension, return_sequences=True, return_state=True,
                                             name='decoder_hidden_1', dropout=dropout)
    # all decoders the initial state is encoder last state of last layer
    decoder_outputs, _ = decoder(embed_dec, initial_state=encoder_states)
    for i in range(2,d_layer+1):
      layer_name = 'decoder_hidden_%d' % i
      decoder = keras.layers.GRU(latent_dimension, return_sequences=True, return_state=True,
                                                 name=layer_name, dropout=dropout)
      decoder_outputs, _ = decoder(decoder_outputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax", name='decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(
          optimizer="rmsprop", loss="categorical_crossentropy",
          metrics=['accuracy'])#, metrics=[my_metric]                 

    # earlystopping = EarlyStopping(
    #     monitor="val_loss", min_delta=0.01, patience=5, verbose=2, mode="min")
    
    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          #callbacks=WandbCallback()
      )
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc = model.get_layer(
              'encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [state_h_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  # input_2
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []

    for j in range(1, n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs, state_h_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec]
        decoder_states_inputs += current_states_inputs
    decoder_dense = model.get_layer('decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model(
        [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
    )
    return encoder_model, decoder_model


  elif cell_type=="LSTM":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,
                                             name='encoder_embedding')(encoder_inputs)
    encoder = keras.layers.LSTM(latent_dimension, return_state=True, return_sequences=True,
                                             name='encoder_hidden_1', dropout=dropout)
    encoder_outputs, state_h, state_c = encoder(embed)
    for i in range(2,e_layer+1):
      layer_name = ('encoder_hidden_%d') % i
      encoder = keras.layers.LSTM(latent_dimension, return_state=True, return_sequences=True,
                                                 name=layer_name, dropout=dropout)
      encoder_outputs, state_h, state_c = encoder(encoder_outputs, initial_state=[state_h,state_c])
    encoder_states = None
    encoder_states = [state_h, state_c]

    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(
        decoder_inputs)
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.LSTM(latent_dimension, return_sequences=True, return_state=True,
                                             name='decoder_hidden_1', dropout=dropout)
    # all decoders the initial state is encoder last state of last layer
    decoder_outputs, _,_ = decoder(embed_dec, initial_state=encoder_states)
    for i in range(2,d_layer+1):
      layer_name = 'decoder_hidden_%d' % i
      decoder = keras.layers.LSTM(latent_dimension, return_sequences=True, return_state=True,
                                                 name=layer_name, dropout=dropout)
      decoder_outputs, _,_ = decoder(decoder_outputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax", name='decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(
          optimizer="rmsprop", loss="categorical_crossentropy",
          metrics=['accuracy'])#, metrics=[my_metric]                 
    
    # earlystopping = EarlyStopping(
    #     monitor="val_loss", min_delta=0.01, patience=5, verbose=2, mode="min")
    
    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          #callbacks=WandbCallback()
      )
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc, state_c_enc = model.get_layer(
              'encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  # input_2
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []

    for j in range(1,n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        decoder_state_input_c = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs, state_h_dec, state_c_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec, state_c_dec]
        decoder_states_inputs += current_states_inputs
    decoder_dense = model.get_layer('decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model(
        [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
    )
    return encoder_model, decoder_model
      

In [ ]:
def decode_sequence(input_seq,n_decoder_layers,cell_type,encoder_model,decoder_model):
        # Encode the input as state vectors.
        states_value = [encoder_model.predict(input_seq)]*n_decoder_layers

        # Generate empty target sequence of length 1.
        empty_seq = np.zeros((1, 1))
        # Populate the first character of target sequence with the start character.
        empty_seq[0, 0] = target_token_index["\t"]
        target_seq = empty_seq

        # Sampling loop for a batch of sequences
        # (to simplify, here we assume a batch of size 1).
        stop_condition = False
        decoded_sentence = ""
        while not stop_condition:
            if cell_type is not None and (cell_type.lower() == 'rnn' or cell_type.lower() == 'gru'):
                temp = decoder_model.predict([target_seq] + [states_value])
                output_tokens, states_value = temp[0], temp[1:]
            else:
                temp = decoder_model.predict([target_seq] + states_value )
                output_tokens, states_value = temp[0], temp[1:]

            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_char = reverse_target_char_index[sampled_token_index]
            decoded_sentence += sampled_char

            # Exit condition: either hit max length
            # or find stop character.
            if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
                stop_condition = True

            # Update the target sequence (of length 1).
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index

        return decoded_sentence

In [ ]:
def accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model, verbose=False):
        n_correct = 0
        n_total = 0
        for seq_index in range(len(val_encoder_input_data)):
            # Take one sequence (part of the training set)
            # for trying out decoding.
            input_seq = val_encoder_input_data[seq_index: seq_index + 1]
            # Generate empty target sequence of length 1.
            # empty_seq = np.zeros((1, 1))
            # # Populate the first character of target sequence with the start character.
            # empty_seq[0, 0] = self.target_token_index["\t"]
            decoded_sentence = decode_sequence(input_seq,n_decoder_layers,'LSTM',encoder_model,decoder_model)

            if decoded_sentence.strip() == val_target_texts[seq_index].strip():
                n_correct += 1

            n_total += 1

            if verbose:
                print('Prediction ', decoded_sentence.strip(), ',Ground Truth ', val_target_texts[seq_index].strip())

        return n_correct * 100.0 / n_total

In [ ]:
# parameters
embedding_size=256
n_encoder_tokens=num_encoder_tokens
n_decoder_tokens=num_decoder_tokens
n_encoder_layers=3
n_decoder_layers=3
latent_dimension=256
cell_type='GRU'
target_token_index=target_token_index
max_decoder_seq_length=max_decoder_seq_length
reverse_target_char_index=reverse_target_char_index
dropout=0.5
epochs=25

In [ ]:
#calling rnn
encoder_model, decoder_model=seq2seq(embedding_size, num_encoder_tokens,num_decoder_tokens,n_encoder_layers, n_decoder_layers,latent_dimension,
                cell_type, target_token_index, max_decoder_seq_length,reverse_target_char_index, dropout ,encoder_input_data, decoder_input_data,
                decoder_target_data,batch_size,epochs)

# val_accuracy= accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model)
# print('Validation accuracy: ', val_accuracy)

# subset = 0
# val_accuracy = accuracy(val_encoder_input_data[0:subset], val_target_texts[0:subset],n_decoder_layers,encoder_model,decoder_model) if subset>0 \
#     else accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model)
# print('Validation accuracy: ', val_accuracy)

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

In [ ]:
def fit():
  config_defaults = {
            "cell_type":'LSTM',
            "num_encoder_layers":2,
            "num_decoder_layers":3,
            "embedding_size":256,
            "latent_dimension":256,
            "dropout":0.2,
            "epochs":25
        }
  wandb.init(config=config_defaults)

  config = wandb.config
  
  cell_type=config.cell_type
  n_encoder_layers=config.num_encoder_layers
  n_decoder_layers=config.num_decoder_layers
  embedding_size=config.embedding_size
  latent_dimension=config.latent_dimension
  dropout=config.dropout
  epochs=config.epochs

  run_name = "cell_type_{}_nel_{}_ndl_{}_drop_{}_emd_{}_ld_{}".format(cell_type, n_encoder_layers, n_decoder_layers, dropout, embedding_size, latent_dimension )
  
  encoder_model, decoder_model=seq2seq(embedding_size, num_encoder_tokens,num_decoder_tokens,n_encoder_layers, n_decoder_layers,latent_dimension,
                cell_type, target_token_index, max_decoder_seq_length,reverse_target_char_index, dropout ,encoder_input_data, decoder_input_data,
                decoder_target_data,batch_size,epochs)
  
  val_accuracy=accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model)
  print("Validation Accuracy:", val_accuracy)
  wandb.log({'val_accuracy': val_accuracy})
  wandb.run.name = run_name
  wandb.run.save()
  wandb.run.finish()
  

In [ ]:
# run sweeps
sweep_config = {
    'method': 'bayes',  # grid, random
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'embedding_size': {
            'values': [64,128,256]
        },
        'num_encoder_layers': {
            'values': [1,2,3]
        },
        'num_decoder_layers': {
            'values': [1,2,3]
        },
        'latent_dimension': {
            'values': [64, 256, 512]
        },
        'cell_type': {
            'values': ['RNN', 'GRU', 'LSTM']
        },                             
        'dropout': {
            'values': [0.3,0.4,0.5,0.0,0.2]
        },
        'epochs': {
            'values': [25,20,30]
        }
    }
}

#sweep_id = wandb.sweep(sweep_config,entity="anandh" ,project="CS6910_Assignment3_S2S")
# wandb.agent(sweep_id, fit, count=10)
sweep_id="5y0u4iyv"
wandb.agent(sweep_id, fit, entity="anandh", project = "CS6910_Assignment3_S2S", count = 6)


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 8unjpy35 with config:
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.3
wandb: 	embedding_size: 128
wandb: 	epochs: 20
wandb: 	latent_dimension: 512
wandb: 	num_decoder_layers: 2
wandb: 	num_encoder_layers: 1
wandb: Currently logged in as: anandh (use `wandb login --relogin` to force relogin)


Epoch 1/20
1066/1066 [==============================] - 34s 26ms/step - loss: 0.7614 - accuracy: 0.7802
Epoch 2/20
1066/1066 [==============================] - 27s 26ms/step - loss: 0.5101 - accuracy: 0.8451
Epoch 3/20
1066/1066 [==============================] - 27s 26ms/step - loss: 0.4124 - accuracy: 0.8726
Epoch 4/20
1066/1066 [==============================] - 27s 26ms/step - loss: 0.3330 - accuracy: 0.8953
Epoch 5/20
1066/1066 [==============================] - 27s 26ms/step - loss: 0.2444 - accuracy: 0.9238
Epoch 6/20
1066/1066 [==============================] - 27s 26ms/step - loss: 0.1791 - accuracy: 0.9447
Epoch 7/20
1066/1066 [==============================] - 27s 26ms/step - loss: 0.1295 - accuracy: 0.9606
Epoch 8/20
1066/1066 [==============================] - 27s 26ms/step - loss: 0.0988 - accuracy: 0.9702
Epoch 9/20
1066/1066 [==============================] - 27s 26ms/step - loss: 0.0791 - accuracy: 0.9762
Epoch 10/20
1066/1066 [==============================] - 27s 26m

wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Validation Accuracy: 12.701435687078817


val_accuracy,▁
val_accuracy,12.70144


...

Epoch 1/25
1066/1066 [==============================] - 43s 33ms/step - loss: 0.7961 - accuracy: 0.7715
Epoch 2/25
1066/1066 [==============================] - 35s 33ms/step - loss: 0.4867 - accuracy: 0.8534
Epoch 3/25
1066/1066 [==============================] - 35s 33ms/step - loss: 0.3543 - accuracy: 0.8904
Epoch 4/25
1066/1066 [==============================] - 35s 33ms/step - loss: 0.2581 - accuracy: 0.9188
Epoch 5/25
1066/1066 [==============================] - 35s 33ms/step - loss: 0.1713 - accuracy: 0.9466
Epoch 6/25
1066/1066 [==============================] - 35s 33ms/step - loss: 0.1116 - accuracy: 0.9657
Epoch 7/25
1066/1066 [==============================] - 35s 33ms/step - loss: 0.0783 - accuracy: 0.9763
Epoch 8/25
1066/1066 [==============================] - 35s 33ms/step - loss: 0.0592 - accuracy: 0.9822
Epoch 9/25
1066/1066 [==============================] - 35s 33ms/step - loss: 0.0469 - accuracy: 0.9858
Epoch 10/25
1066/1066 [==============================] - 35s 33m

val_accuracy,▁
val_accuracy,8.54087


wandb: Agent Starting Run: va0p15cl with config:
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.5
wandb: 	embedding_size: 64
wandb: 	epochs: 30
wandb: 	latent_dimension: 512
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 3


Epoch 1/30
1066/1066 [==============================] - 66s 52ms/step - loss: 0.7915 - accuracy: 0.7733
Epoch 2/30
1066/1066 [==============================] - 55s 52ms/step - loss: 0.3504 - accuracy: 0.8963
Epoch 3/30
1066/1066 [==============================] - 55s 52ms/step - loss: 0.1581 - accuracy: 0.9555
Epoch 4/30
1066/1066 [==============================] - 55s 52ms/step - loss: 0.0956 - accuracy: 0.9735
Epoch 5/30
1066/1066 [==============================] - 55s 52ms/step - loss: 0.0671 - accuracy: 0.9813
Epoch 6/30
1066/1066 [==============================] - 55s 52ms/step - loss: 0.0510 - accuracy: 0.9856
Epoch 7/30
1066/1066 [==============================] - 55s 52ms/step - loss: 0.0402 - accuracy: 0.9884
Epoch 8/30
1066/1066 [==============================] - 55s 52ms/step - loss: 0.0335 - accuracy: 0.9902
Epoch 9/30
1066/1066 [==============================] - 55s 52ms/step - loss: 0.0286 - accuracy: 0.9915
Epoch 10/30
1066/1066 [==============================] - 55s 52m

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
#####
encoder_model, decoder_model=seq2seq(256, num_encoder_tokens,num_decoder_tokens,
                2, 3,latent_dim,
                'LSTM', target_token_index, max_decoder_seq_length,
                reverse_target_char_index, 0.2,encoder_input_data, decoder_input_data,
                decoder_target_data,batch_size)

val_accuracy= accuracy(val_encoder_input_data, val_target_texts,3,encoder_model,decoder_model)
print('Validation accuracy: ', val_accuracy)

Epoch 1/2
1066/1066 [==============================] - 27s 19ms/step - loss: 0.7256 - accuracy: 0.7928
Epoch 2/2
1066/1066 [==============================] - 20s 19ms/step - loss: 0.2887 - accuracy: 0.9155
Validation accuracy:  21.725754468209786


In [ ]:
def beam_search(input_seq,encoder_model,decoder_model, beam_size,n_decoder_layers,cell_type):
        sequences = [([target_token_index["\t"]], 0.0)]
        # Encode the input as state vectors.
        states_value = [[encoder_model.predict(input_seq)]*n_decoder_layers]

        stop_condition = False
        t = 0
        while not stop_condition:
            all_seq = list()
            char_sequences = []
            for seq, score in sequences:
                char_seq = ''
                for index in seq:
                    char_seq += reverse_target_char_index[index]
                char_sequences.append((char_seq, score))
            #print('at time ', t, char_sequences)
            t += 1
            for i in range(len(sequences)):
                seq, score = sequences[i]
                if seq[-1] == target_token_index["\n"] or seq[-1] == target_token_index[" "]:
                    all_seq.append((seq, score))
                    continue
                target_seq = np.zeros((1, 1))
                target_seq[0, 0] = seq[-1]
                # print('target seq', seq[-1], self.reverse_target_char_index[seq[-1]])
                if cell_type is not None and (cell_type.lower() == 'rnn' or cell_type.lower() == 'gru'):
                    temp = decoder_model.predict([target_seq] + [states_value[i]])
                    output_tokens, temp_states = temp[0], temp[1:]
                else:
                    temp = decoder_model.predict([target_seq] + states_value[i] )
                    output_tokens, temp_states = temp[0], temp[1:]

                if t == 1:
                    states_value = [temp_states] * beam_size
                else:
                    states_value[i] = temp_states

                for j in range(len(output_tokens[0, -1, :])):
                    candidate = (seq + [j], score - math.log(output_tokens[0, -1, j]))
                    all_seq.append(candidate)

                # Exit condition: either hit max length
                # or find stop character.
                sampled_token_index = np.argmax(output_tokens[0, -1, :])
                sampled_char = reverse_target_char_index[sampled_token_index]
                # print('prob', output_tokens[0, -1, :])
                # print('sampledchar ', sampled_char)

            sorted_by_prob = sorted(all_seq, key=lambda tup: tup[1])

            # print all possible sequences
            char_sequences = []
            for seq, score in sequences:
                char_seq = ''
                for index in seq:
                    char_seq += reverse_target_char_index[index]
                char_sequences.append((char_seq, score))
            # print('Printing all sequences')
            # print(char_sequences)

            # select the top k sequences
            sequences = sorted_by_prob[:beam_size]
            if t > max_decoder_seq_length:
                stop_condition = True
            # if every sequence has predicted \n we should stop
            all_seq_ended = True
            for seq, _ in sequences:
                if seq[-1] != target_token_index["\n"]:
                    all_seq_ended = False
                    break
            if all_seq_ended:
                stop_condition = True
        # create character out of indexes
        char_sequences = []
        for seq, score in sequences:
            char_seq = ''
            for index in seq:
                char_seq += reverse_target_char_index[index]
            char_sequences.append((char_seq, score))
        return char_sequences

In [ ]:
encoder_model, decoder_model=seq2seq(embedding_size, num_encoder_tokens,num_decoder_tokens,n_encoder_layers, n_decoder_layers,latent_dimension,
                cell_type, target_token_index, max_decoder_seq_length,reverse_target_char_index, dropout ,encoder_input_data, decoder_input_data,
                decoder_target_data,batch_size,epochs)
# compute test accuracy
print('Reading test data')
test_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.test.tsv"
# open and save the files to lists
with open(test_data, "r", encoding="utf-8") as f:
    test_lines = f.read().split("\n")
# popping the last element of all the lists since it is empty character
test_lines.pop()
# embedding test
# for test data, almost same
test_input_texts = []
test_target_texts = []
for line in test_lines[: (len(test_lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    test_input_texts.append(input_text)
    test_target_texts.append(target_text)
test_max_encoder_seq_length = max([len(txt) for txt in test_input_texts])
test_max_decoder_seq_length = max([len(txt) for txt in test_target_texts])
test_encoder_input_data = np.zeros(
    (len(test_input_texts), test_max_encoder_seq_length), dtype="float32"
)
test_decoder_input_data = np.zeros(
    (len(test_input_texts), test_max_decoder_seq_length), dtype="float32"
)
test_decoder_target_data = np.zeros(
    (len(test_input_texts), test_max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
for i, (input_text, target_text) in enumerate(zip(test_input_texts, test_target_texts)):
    for t, char in enumerate(input_text):
        test_encoder_input_data[i, t] = input_token_index[char]
    test_encoder_input_data[i, t + 1:] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        test_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            test_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    test_decoder_input_data[i, t + 1:] = target_token_index[" "]
    test_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

print('Calculating test accuracy')
test_accuracy = {}
for beamSize in range(1,4):
  df = pd.DataFrame(columns=['Source', 'Predictions', 'GroundTruth'])
  n_correct = 0
  n_total = 0
  for seq_index in range(len(test_encoder_input_data)):
      decoded_sentence = beam_search(test_encoder_input_data[seq_index:seq_index+1],encoder_model,decoder_model,beamSize,n_decoder_layers,cell_type)

      if test_target_texts[seq_index].strip() == decoded_sentence[0][0].strip():
          n_correct += 1

      n_total += 1
      row = {}
      row['SourceText'] = test_input_texts[seq_index].strip()
      row['GroundTruth'] = test_target_texts[seq_index].strip()
      row['Prediction'] = decoded_sentence[0][0].strip()
      df = df.append(row, ignore_index=True)
  df.to_csv('predictions_'+str(beamSize)+'.csv', index=False)  
  test_accuracy[beamSize] = (n_correct * 100.0 / n_total)
print('Test accuracy ', test_accuracy)

import time
for beamSize in range(1,4):
  files.download('predictions_'+str(beamSize)+'.csv')
  time.sleep(30)

Epoch 1/25
1066/1066 [==============================] - 36s 25ms/step - loss: 0.6057 - accuracy: 0.8263
Epoch 2/25
1066/1066 [==============================] - 26s 25ms/step - loss: 0.2378 - accuracy: 0.9320
Epoch 3/25
1066/1066 [==============================] - 26s 25ms/step - loss: 0.1513 - accuracy: 0.9574
Epoch 4/25
1066/1066 [==============================] - 26s 25ms/step - loss: 0.1188 - accuracy: 0.9666
Epoch 5/25
1066/1066 [==============================] - 26s 25ms/step - loss: 0.1010 - accuracy: 0.9716
Epoch 6/25
1066/1066 [==============================] - 26s 25ms/step - loss: 0.0896 - accuracy: 0.9747
Epoch 7/25
1066/1066 [==============================] - 26s 25ms/step - loss: 0.0811 - accuracy: 0.9768
Epoch 8/25
1066/1066 [==============================] - 26s 24ms/step - loss: 0.0747 - accuracy: 0.9786
Epoch 9/25
1066/1066 [==============================] - 26s 25ms/step - loss: 0.0697 - accuracy: 0.9797
Epoch 10/25
1066/1066 [==============================] - 26s 24m

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>